In [18]:
# This notebook is used to write the scripts and then write them into shell scripts - isn't required for the pipeline

# GIF: Nicholas Cage Montage
---
Combine a sequence of images of Nicholas Cage's face, sorted by smartgrid

In [ ]:
# Goal ====================
# Directories:
    # temp - temp files
    # inputs - source images - 
    # outputs - processed images in sequence
    # movies - final movie gif
#  Scripts:
    # build-directories.sh - script to create folders
    # run-cage.sh - used to run the pipeline as outlined below
        # get-cage.sh - scrapes images of nicholas cage and puts them into inputs
        # process-cage.sh - goes through input images and filters out non-cage photos + resizes to 440 X 220
        # sort-cage.sh - use smartgrid to sort images and put them into outputs
        # composite-cage - use smartgrid to sort images and put them into outputs    

In [28]:
# Build Directories ===================
build_dirs = 'mkdir temp inputs outputs movies'
! {build_dirs}

In [63]:
# Misc. helpers
def write_script(title, content) :
    f = open('{}.sh'.format(title), 'w')
    f.write('set -ex\n{}'.format(content))
    f.close()

In [123]:
# Get Nick Cage photos ================
# source: http://niccageaseveryone.blogspot.co.nz

get_cage = 'for PAGE in $(seq -f "%02g" 1 12)\
\ndo\
\n# build the url \
\nURL="http://niccageaseveryone.blogspot.co.nz/2010/"$PAGE\
\n\techo $URL\
\n\t# fetch the images \\\
\n\twget \\\
\n\t\t--adjust-extension \\\
\n\t\t--no-directories \\\
\n\t\t--convert-links \\\
\n\t\t--backup-converted \\\
\n\t\t--random-wait\\\
\n\t\t--limit-rate=100k \\\
\n\t\t--span-hosts \\\
\n\t\t--directory-prefix=temp/downloads \\\
\n\t\t--page-requisites \\\
\n\t\t--timestamping \\\
\n\t\t--execute robots=off \\\
\n\t\t--accept=*.jpg \\\
\n\t\t$URL\
\ndone'

write_script("get-cage", get_cage)